# Modelagem de Reclamações de Aeroporto

Treinamento e avaliação de modelos de Machine Learning para predizer reclamações com maior probabilidade de serem negadas.

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, make_scorer, recall_score
import joblib
import os

## Carregamento dos dados processados

Leitura dos conjuntos de treino processados (`X_train`, `y_train`) para modelagem.

In [ ]:
X_train = pd.read_csv("data/processed/X_train.csv")
y_train = pd.read_csv("data/processed/y_train.csv").values.ravel() 

## Função de treinamento

Função para realizar GridSearchCV e validação cruzada, retornando o melhor modelo, o score médio e os melhores parâmetros.

In [ ]:
recall_macro = make_scorer(recall_score, average='macro')

def train_classifier(model, param_grid, X, y, cv=5):
    grid = GridSearchCV(
        model,
        param_grid,
        cv=cv,
        scoring=recall_macro,
        n_jobs=-1
    )
    
    grid.fit(X, y)
    best_model = grid.best_estimator_
    
    scores = cross_val_score(best_model, X, y, scoring=recall_macro, cv=cv, n_jobs=-1)
    mean_cv_recall = scores.mean()
    
    return best_model, mean_cv_recall, grid.best_params_


## Definição de parâmetros dos modelos

Configuração de hiperparâmetros para Decision Tree, Random Forest e Logistic Regression, com foco em reduzir overfitting e equilibrar classes.

In [ ]:
dt_params = {
    'max_depth': [3, 5, 8, 10],             
    'min_samples_split': [5, 10, 20],       
    'min_samples_leaf': [2, 4, 6, 8],       
    'max_features': ['sqrt', 'log2'],       
    'class_weight': ['balanced']
}

rf_params = {
    'n_estimators': [100, 200],            
    'max_depth': [8, 10, 12],              
    'min_samples_split': [5, 10, 20],       
    'min_samples_leaf': [2, 4, 6],         
    'max_features': ['sqrt', 'log2'],       
    'bootstrap': [True],
    'class_weight': ['balanced'],
    'max_samples': [0.7, 0.8, 0.9]          
}


logreg_params = {
    'penalty': ['l2'],                    
    'C': [0.001, 0.01, 0.1, 1, 10],       
    'solver': ['lbfgs'],
    'max_iter': [1000],                     
    'class_weight': ['balanced']
}



## Treinamento dos modelos

Execução do treinamento e validação cruzada para encontrar os melhores modelos e parâmetros.

In [63]:
best_dt, f1_dt, best_params_dt = train_classifier(DecisionTreeClassifier(random_state=42), dt_params, X_train, y_train)
best_rf, f1_rf, best_params_rf = train_classifier(RandomForestClassifier(random_state=42), rf_params, X_train, y_train)
best_logreg, f1_logreg, best_params_logreg = train_classifier(LogisticRegression(random_state=42), logreg_params, X_train, y_train)

print("Decision Tree - Melhores parâmetros:", best_params_dt, "| Score CV:", f1_dt)
print("Random Forest - Melhores parâmetros:", best_params_rf, "| Score CV:", f1_rf)
print("Logistic Regression - Melhores parâmetros:", best_params_logreg, "| Score CV:", f1_logreg)


Decision Tree - Melhores parâmetros: {'class_weight': 'balanced', 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 10} | Score CV: 0.5967328162523529
Random Forest - Melhores parâmetros: {'bootstrap': True, 'class_weight': 'balanced', 'max_depth': 8, 'max_features': 'sqrt', 'max_samples': 0.9, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 200} | Score CV: 0.6154718634386966
Logistic Regression - Melhores parâmetros: {'C': 10, 'class_weight': 'balanced', 'max_iter': 1000, 'penalty': 'l2', 'solver': 'lbfgs'} | Score CV: 0.5779820049328228


## Avaliação dos modelos no conjunto de treino

Geração de classification reports e matrizes de confusão para verificar o desempenho de cada modelo no treino.

In [64]:
# Decision Tree
y_pred_dt = best_dt.predict(X_train)
print("Decision Tree Classification Report (Treino):\n", classification_report(y_train, y_pred_dt))
print("Decision Tree Confusion Matrix (Treino):\n", confusion_matrix(y_train, y_pred_dt))

# Random Forest
y_pred_rf = best_rf.predict(X_train)
print("Random Forest Classification Report (Treino):\n", classification_report(y_train, y_pred_rf))
print("Random Forest Confusion Matrix (Treino):\n", confusion_matrix(y_train, y_pred_rf))

# Logistic Regression
y_pred_logreg = best_logreg.predict(X_train)
print("Logistic Regression Classification Report (Treino):\n", classification_report(y_train, y_pred_logreg))
print("Logistic Regression Confusion Matrix (Treino):\n", confusion_matrix(y_train, y_pred_logreg))



Decision Tree Classification Report (Treino):
               precision    recall  f1-score   support

           0       0.75      0.38      0.50      2481
           1       0.58      0.87      0.70      2481

    accuracy                           0.63      4962
   macro avg       0.66      0.63      0.60      4962
weighted avg       0.66      0.63      0.60      4962

Decision Tree Confusion Matrix (Treino):
 [[ 944 1537]
 [ 322 2159]]
Random Forest Classification Report (Treino):
               precision    recall  f1-score   support

           0       0.78      0.41      0.54      2481
           1       0.60      0.88      0.72      2481

    accuracy                           0.65      4962
   macro avg       0.69      0.65      0.63      4962
weighted avg       0.69      0.65      0.63      4962

Random Forest Confusion Matrix (Treino):
 [[1027 1454]
 [ 291 2190]]
Logistic Regression Classification Report (Treino):
               precision    recall  f1-score   support

      

## Comparação entre valores reais e predições

Criação de DataFrames para comparar os valores reais da variável alvo com as predições de cada modelo.

In [65]:
# Comparar y_real e y_pred


# Decision Tree
comparison_dt = pd.DataFrame({
    'y_real': y_train,
    'y_pred': y_pred_dt
})
print("Decision Tree - Comparação alvo x predição (treino):")
print(comparison_dt.head(10))

# Random Forest
comparison_rf = pd.DataFrame({
    'y_real': y_train,
    'y_pred': y_pred_rf
})
print("Random Forest - Comparação alvo x predição (treino):")
print(comparison_rf.head(10))

# Logistic Regression
comparison_logreg = pd.DataFrame({
    'y_real': y_train,
    'y_pred': y_pred_logreg
})
print("Logistic Regression - Comparação alvo x predição (treino):")
print(comparison_logreg.head(10))


Decision Tree - Comparação alvo x predição (treino):
   y_real  y_pred
0       1       1
1       1       1
2       0       1
3       1       1
4       0       0
5       0       0
6       0       0
7       1       1
8       0       1
9       0       1
Random Forest - Comparação alvo x predição (treino):
   y_real  y_pred
0       1       1
1       1       1
2       0       0
3       1       1
4       0       0
5       0       0
6       0       1
7       1       1
8       0       1
9       0       1
Logistic Regression - Comparação alvo x predição (treino):
   y_real  y_pred
0       1       1
1       1       1
2       0       0
3       1       1
4       0       0
5       0       0
6       0       1
7       1       0
8       0       0
9       0       0


## Salvamento dos modelos

Armazenamento dos modelos treinados em arquivos `.pkl` para uso futuro.

In [66]:
import os
import joblib

os.makedirs("models", exist_ok=True)

joblib.dump(best_dt, "models/decision_tree_model.pkl")
joblib.dump(best_rf, "models/random_forest_model.pkl")
joblib.dump(best_logreg, "models/logreg_model.pkl")


['models/logreg_model.pkl']